In [5]:
import sys
sys.path.append('../Classes')
from AzureSearch import Config
import pandas as pd
import openai
import json
from azure.identity import DefaultAzureCredential

config = Config(path="../Classes/environment.env")
cred = DefaultAzureCredential(exclude_interactive_browser_credential=False)
token = cred.get_token("https://cognitiveservices.azure.com/.default")

openai.api_key = token.token
openai.api_version = config.openai_api_version
openai.api_type = "azure_ad"
openai.api_base = config.openai_api_base

functions = [
    {
        "name": "rate_generated_answer",
        "description": """Use to send the rating of the generated answer in comparison to ground truth. Remember that a correct answer can be written with different words, and have the same meaning as the groun truth.""",
        "parameters": {
            "type": "object",
            "properties": {
                "rating": {
                    "type": "string",
                    "description": "The Rating of the generated answer",
                    "enum": ["1", "2", "3", "4", "5"]
                }
            },
            "required": ["rating"]
        }
    }
]

def rate_answer(prompt, generated_answer, ground_truth):
    """
    Use the OpenAI API to rate the generated answer against the ground truth.
    This function constructs a message object that includes the prompt, generated answer, and ground truth.
    Then, it requests the model to score the generated answer.
    """
    messages = [
        {"role": "system", "content": f"Prompt: {prompt}"},
        {"role": "user", "content": f"Generated Answer: {generated_answer}\nGround Truth: {ground_truth}"}
    ]

    response = openai.ChatCompletion.create(
        engine=config.gpt,
        messages=messages,
        temperature=0,
        functions=functions,
        function_call={"name": "rate_generated_answer"},
    )

    # Extract and return the model's rating
    dirty_class = response['choices'][0]['message']['function_call']['arguments']
    rating = json.loads(dirty_class).get("rating")
    return rating

def process_and_rate_answers(prompts_csv, answers_csv):
    """
    Processes the CSV files with prompts, generated answers, and ground truths.
    Calls the rate_answer function for each combination of prompt and answer row and collects the ratings.
    """
    prompts_df = pd.read_csv(prompts_csv)
    answers_df = pd.read_csv(answers_csv)

    ratings = []

    for prompt_index, prompt_row in prompts_df.iterrows():
        for answer_index, answer_row in answers_df.iterrows():
            prompt = prompt_row['prompt']
            generated_answer = answer_row['generated']
            ground_truth = answer_row['ground']

            rating = rate_answer(prompt, generated_answer, ground_truth)

            ratings.append({
                'prompt_index': prompt_index,
                'prompt': prompt,
                'generated_answer': generated_answer,
                'ground_truth': ground_truth,
                'rating': rating
            })

    # Create a DataFrame from the ratings
    ratings_df = pd.DataFrame(ratings)

    return ratings_df

# Example call
ratings_df = process_and_rate_answers('./prompts.csv', './answers.csv')
ratings_df.to_excel('./Rated_Answers.xlsx')
ratings_df

,prompt_index,prompt,generated_answer,ground_truth,rating
0,0,System:\nYou are an AI assistant. You will be ...,### Opening a Savings Account Steps: \n- Go to...,### To open a new savings account: \n1. Visit ...,3
1,0,System:\nYou are an AI assistant. You will be ...,### Mutual Fund Investment: \n- Access interne...,### To invest in a mutual fund: \n1. Log in to...,2
2,0,System:\nYou are an AI assistant. You will be ...,### Credit Score Viewing: \n- Open bank app \n...,### To check your credit score: \n1. Log into ...,3
3,0,System:\nYou are an AI assistant. You will be ...,### International Transfer: \n- Login to bank ...,### To transfer funds internationally: \n1. Lo...,2
4,0,System:\nYou are an AI assistant. You will be ...,### Transaction Dispute: \n- Simply call bank ...,### To dispute a transaction: \n1. Call the ba...,2
